# Multi-GPU

In this notebook, we learn how to train a model with multiple devices in TensorFlow.

In machine learning, there are two types of parallelism:

**Data parallelism**: A single model is replicated on multiple devices, each processing different batches of data, and the gradients obtained from backprop are merged before updating the model weights. 

**Model parallelism**: Different parts of a single model run on different devices. This is used for very large models (such as LLMs).

We will consider both in the following sections.

**Note:** for this example, if we are using colab, we need to downgrade the tensorflow version as there is a bug with the most recent version. 

In [1]:
!pip install tensorflow==2.10.1

In [2]:
import tensorflow as tf
import tensorflow.keras as keras

2023-09-15 14:41:15.398516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 14:41:15.571649: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 14:41:15.611667: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-15 14:41:16.368769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Create virtual GPUs

In this training course, we may not able to provide multiple physical GPUs to everyone. Therefore, we use virtual or logical GPUs instead. TensorFlow will see these virtual GPUs as logically independent, so the code in the later sections should also work on multiple physical GPUs.

Now we create 4 virtual GPUs, each taking 1GB memory.

In [3]:
# specify number of GPUs
N_GPUS = 4
MEM_GPU = 1024

physical_gpus = tf.config.list_physical_devices('GPU')
tf.config.set_logical_device_configuration(
        physical_gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=MEM_GPU)] * N_GPUS)

logical_gpus = tf.config.list_logical_devices('GPU')

2023-09-15 14:41:17.381644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-15 14:41:17.421038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-15 14:41:17.421274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-15 14:41:17.423433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [4]:
# print the GPUs
print('Physical GPUs:')
for device in physical_gpus:
    print(device)

print('\nLogical GPUs:')
for device in logical_gpus:
    print(device)

Physical GPUs:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Logical GPUs:
LogicalDevice(name='/device:GPU:0', device_type='GPU')
LogicalDevice(name='/device:GPU:1', device_type='GPU')
LogicalDevice(name='/device:GPU:2', device_type='GPU')
LogicalDevice(name='/device:GPU:3', device_type='GPU')


## Data parallelism

First, we define the functions to create the model and datasets. If you are unsure about any lines, please revisit `DNN/DNN_basics.ipynb`.

In [5]:
def get_compiled_model():
    # Make a simple 4-layer densely-connected neural network.
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
    x = keras.layers.Dense(256, activation="relu")(x)
    outputs = keras.layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [6]:
def get_dataset():
    batch_size = 256
    num_val_samples = 10000

    # Return the MNIST dataset in the form of a `tf.data.Dataset`.
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Preprocess the data (these are Numpy arrays)
    x_train = x_train.reshape(-1, 784).astype("float32") / 255
    x_test = x_test.reshape(-1, 784).astype("float32") / 255
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")

    # Reserve num_val_samples samples for validation
    x_val = x_train[-num_val_samples:]
    y_val = y_train[-num_val_samples:]
    x_train = x_train[:-num_val_samples]
    y_train = y_train[:-num_val_samples]
    
    # datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    
    # disable auto-share policy for a tensorflow issue. This may be fixed in the future.
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
    train_dataset = train_dataset.with_options(options)
    val_dataset = val_dataset.with_options(options)
    test_dataset = test_dataset.with_options(options)
    return train_dataset, val_dataset, test_dataset

Next, from `tf.distribute`, we create a strategy for parallelism. Here we use `MirroredStrategy`, which is for synchronous training across multiple replicas on one machine. For distributed training on multiple machines, one needs `MultiWorkerMirroredStrategy` and select devices from the machines.


In [7]:
# devices=None will use all avialable GPUs; 
# devices=['GPU:0', 'GPU:1'] will use two GPUs
strategy = tf.distribute.MirroredStrategy(devices=None)
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4


Now we create the model within the scope of the `MirroredStrategy`. Note that **this is the only difference from the single GPU case**. 

In [8]:
with strategy.scope():
    model = get_compiled_model()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

Now start training. 

In [9]:
# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/j

[0.1123083233833313, 0.9657999873161316]

## Model parallelism

In this section, we demonstrate model parallelism by manually assigning each layer a different device. 

In [10]:
# input and first hidden on GPU:0
with tf.device("GPU:0"):
    inputs = keras.Input(shape=(784,))
    x = keras.layers.Dense(256, activation="relu")(inputs)
# second hidden on GPU:1
with tf.device("GPU:1"):
    x = keras.layers.Dense(256, activation="relu")(x)
# third hidden on GPU:2
with tf.device("GPU:2"):
    x = keras.layers.Dense(256, activation="relu")(x)
# output on GPU:3
with tf.device("GPU:3"):
    outputs = keras.layers.Dense(10)(x)



model = keras.Model(inputs, outputs)
model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

Now start training:

In [11]:
# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

Epoch 1/2
196/196 [==============================] - 1s 5ms/step - loss: 0.3619 - sparse_categorical_accuracy: 0.8966 - val_loss: 0.1375 - val_sparse_categorical_accuracy: 0.9615
Epoch 2/2
40/40 [==============================] - 0s 3ms/step - loss: 0.1021 - sparse_categorical_accuracy: 0.9670


[0.10211756080389023, 0.9670000076293945]

Final notes:
1. On multiple machines, the GPUs can be selected with full paths with machine names. 
2. Overheads for data transfer between physical GPUs can be greatly reduced by new communication hardware technologies such as NVLink and NVSwitch.
3. Model distribution is more common in LLMs. Luckily, `HuggingFace` provides automatic device mapping based on layer sizes and available devices, so one usually does not need to configure model distribution manually.